In [1]:
%pip install pandas python-dotenv pyairtable neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ดึงข้อมูลจาก airtable
import os
import pandas as pd
from dotenv import load_dotenv
from pyairtable import Table

load_dotenv()

api_key = os.getenv("AIRTABLE_API_KEY")
base_id = os.getenv("AIRTABLE_BASE_ID")
table = Table(api_key, base_id, "Table 1")

records = table.all() 

df = pd.DataFrame([r['fields'] for r in records])
display(df.head()) 

C:\Users\MSI\AppData\Local\Temp\ipykernel_22816\3932076754.py:11: DeprecationWarning: Passing API keys or base IDs to pyairtable.Table is deprecated; use Api.table() or Base.table() instead. See https://pyairtable.rtfd.org/en/latest/migrations.html for details.
  table = Table(api_key, base_id, "Table 1")


,Video URL,Caption,Cover Image,Duration,Author Name,Followers,Views,Likes,Shares,Saves,Date Posted,Brand,Category
0,https://www.tiktok.com/@mintps_/video/73866050...,🐈🌤️🤍✨ #ootd #hellokitty #fitcheck #outfitideas,https://p16-common-sign.tiktokcdn-us.com/tos-a...,16,mintps_,70400,776700,90400,3271,11823,2024-07-01,Hello Kitty,เสื้อผ้า
1,https://www.tiktok.com/@mintps_/video/74003283...,Replying to @lyntttttt เจอคำถามเรื่องหน้าม้าบ่...,https://p16-common-sign.tiktokcdn-us.com/tos-a...,31,mintps_,70400,1200000,97900,2683,27855,2024-08-07,mintps_,การจัดแต่งทรงผม
2,https://www.tiktok.com/@mintps_/video/73836327...,My youth is yours @EXO 🤍👑\n\n#2024BAEKHYUNASIA...,https://p16-common-sign.tiktokcdn-eu.com/tos-a...,21,mintps_,70400,334500,61000,1780,3281,2024-06-23,EXO,คอนเสิร์ต
3,https://www.tiktok.com/@muknutkamon/video/7445...,ทำให้ดูแล้ว และถ้าไม่ดู เราจะรู้จักกันแน่ #fyp,https://p16-common-sign.tiktokcdn-us.com/tos-a...,102,muknutkamon,226500,4500000,326000,12600,129920,2024-12-07,muknutkamon,เนื้อหา
4,https://www.tiktok.com/@lookhmee.review/video/...,ลิปใหม่สวยติดทนม้ากแม่!!!! 02 คือต้องตำ #ลิปod...,https://p16-common-sign.tiktokcdn-us.com/tos-m...,22,lookhmee.review,33100,773500,90200,3374,9442,2024-04-27,odbo,ลิปทินท์


In [3]:
# ไม่ต้องโหลดไฟล์ CSV ใหม่แล้ว เพราะเรามีตัวแปร df จากช่องบนสุด
# df = pd.read_csv('raw_influencer_data.csv')  <-- บรรทัดนี้คือสาเหตุ Error ลบทิ้งได้เลย

# 1. สร้าง Dictionary สำหรับ Mapping (เหมือนเดิม)
category_map = {
    'ลิปสติก': 'เครื่องสำอาง',
    'รองพื้น': 'เครื่องสำอาง',
    'คาเฟ่': 'อาหารและเครื่องดื่ม',
    'รีวิวร้านอาหาร': 'อาหารและเครื่องดื่ม'
}

# 2. เปลี่ยนชื่อหมวดหมู่ (แก้ชื่อคอลัมน์ให้ตรงกับ Airtable คือ 'Category')
# ใช้ .map แล้วแทนที่ค่าเดิมใน df เลย
if 'Category' in df.columns:
    df['Category'] = df['Category'].map(category_map).fillna(df['Category'])

# 3. รวมกลุ่มข้อมูล (Grouping)
# แก้ชื่อคอลัมน์ให้ตรงกับ Airtable: 'Author Name', 'Category', 'Views', ...
# เพิ่ม 'Brand' เข้าไปในกลุ่มด้วย ข้อมูลจะได้ไม่หาย
required_columns = ['Author Name', 'Brand', 'Category', 'Views', 'Likes', 'Shares', 'Saves', 'Followers']

# ตรวจสอบว่ามีคอลัมน์ครบไหมก่อนทำงาน
if set(required_columns).issubset(df.columns):
    df_cleaned = df.groupby(['Author Name', 'Brand', 'Category']).agg({
        'Views': 'sum',
        'Likes': 'sum',
        'Shares': 'sum',
        'Saves': 'sum',
        'Followers': 'max' # followers ใช้ค่าสูงสุด
    }).reset_index()

    # 4. บันทึกไฟล์เตรียมนำเข้า Neo4j (ถ้าต้องการเก็บไฟล์ไว้)
    df_cleaned.to_csv('cleaned_data_for_neo4j.csv', index=False)
    
    print("✅ จัดการข้อมูลและบันทึกไฟล์ CSV สำเร็จ!")
    display(df_cleaned.head())
else:
    print("❌ ชื่อคอลัมน์ไม่ตรง ตรวจสอบชื่อคอลัมน์ด้วยคำสั่ง df.columns")
    print(df.columns)

✅ จัดการข้อมูลและบันทึกไฟล์ CSV สำเร็จ!


,Author Name,Brand,Category,Views,Likes,Shares,Saves,Followers
0,bethanyreview,The originote,มาส์กหน้า,1600000,66800,841,6093,16200
1,bethanyreview,cintageshop,เสื้อกันหนาว,486800,71700,911,9214,16200
2,bethanyreview,dazzleme,แป้ง,9600000,102700,7573,21462,16200
3,bethanyreview,dazzleme,แป้งแต่งหน้า,5200000,254700,20800,50443,16200
4,bethanyreview,in2it,ไพรเมอร์เบลอรูขุมขน,1300000,78000,522,6967,16200


In [4]:
from neo4j import GraphDatabase
import os
import pandas as pd  # เรียกใช้ pandas เพื่อจัดการค่าว่าง
from dotenv import load_dotenv

# 1. โหลดค่า Key และตั้งค่าการเชื่อมต่อ (แบบ +ssc ที่เราทำสำเร็จเมื่อกี้)
load_dotenv(override=True)
uri = os.getenv("NEO4J_URI", "").strip()
user = os.getenv("NEO4J_USER", "").strip()
password = os.getenv("NEO4J_PASSWORD", "").strip()

print(f"📍 กำลังเตรียมอัปโหลดไปที่: {uri}")

def upload_to_neo4j(df):
    # --- ส่วนแก้ไข: ล้างข้อมูลก่อนอัปโหลด ---
    # ถ้ามีช่องไหนเป็นค่าว่าง (NaN) ให้ใส่คำว่า "Unknown" หรือเลข 0 แทน
    # เพื่อป้องกัน Error: NaN property value
    df_clean = df.fillna({
        'Author Name': 'Unknown',
        'Brand': 'Unknown',
        'Category': 'Unknown',
        'Video URL': '',
        'Followers': 0,
        'Views': 0,
        'Likes': 0,
        'Shares': 0,
        'Saves': 0
    })
    
    print("🧹 ล้างข้อมูลเรียบร้อย (กำจัด NaN)...")
    print("⏳ กำลังเริ่มอัปโหลด...")

    try:
        # เชื่อมต่อ
        driver = GraphDatabase.driver(uri, auth=(user, password))
        driver.verify_connectivity()
        
        with driver.session() as session:
            for index, row in df_clean.iterrows():
                query = """
                MERGE (i:Influencer {name: $author}) 
                SET i.followers = toInteger($followers)

                MERGE (cat:Category {name: $category}) 
                MERGE (b:Brand {name: $brand})
                MERGE (b)-[be:BELONGS_TO]->(cat)

                MERGE (i)-[r:POSTED_ABOUT]->(b)
                SET r.views = toInteger($views),
                    r.likes = toInteger($likes),
                    r.shares = toInteger($shares),
                    r.saves = toInteger($saves),
                    r.url = $video
                """
                session.run(query, 
                            author=str(row['Author Name']), 
                            brand=str(row['Brand']), 
                            category=str(row['Category']),
                            followers=row['Followers'],
                            views=row['Views'],
                            likes=row['Likes'],
                            shares=row['Shares'],
                            saves=row['Saves'],
                            video=str(row['Video URL']))
                            
        driver.close()
        print("\n🎉🎉🎉 ไชโย! Upload ข้อมูลเสร็จสมบูรณ์เรียบร้อยแล้วครับ!")
        print("👉 ขั้นตอนต่อไป: กลับไปรีเฟรชหน้าเว็บของคุณได้เลย กราฟมาแน่นอน!")

    except Exception as e:
        print(f"\n❌ เกิดข้อผิดพลาด: {e}")

# สั่งรันฟังก์ชัน
if 'df' in locals():
    # เรียกใช้ df (จากช่องโหลดข้อมูลด้านบน)
    upload_to_neo4j(df) 
elif 'df_cleaned' in locals():
    # หรือถ้าคุณใช้ตัวแปรชื่อ df_cleaned
    upload_to_neo4j(df_cleaned)
else:
    print("❌ ไม่พบตัวแปรข้อมูล (df) กรุณากดย้อนไปรันช่องโหลดข้อมูล CSV/Airtable ด้านบนก่อนนะครับ")

📍 กำลังเตรียมอัปโหลดไปที่: neo4j+ssc://074cd7e1.databases.neo4j.io
🧹 ล้างข้อมูลเรียบร้อย (กำจัด NaN)...
⏳ กำลังเริ่มอัปโหลด...

❌ เกิดข้อผิดพลาด: Failed to DNS resolve address 074cd7e1.databases.neo4j.io:7687: [Errno 11001] getaddrinfo failed


In [5]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

# โหลดค่าใหม่
load_dotenv(override=True)

# อ่านค่า URI ที่แก้แล้ว
uri = os.getenv("NEO4J_URI", "").strip()
user = os.getenv("NEO4J_USER", "").strip()
password = os.getenv("NEO4J_PASSWORD", "").strip()

print(f"📍 กำลังลองเชื่อมต่อแบบผ่อนปรน (SSC)...")
print(f"🔗 URI: {uri}") 

try:
    # ลองเชื่อมต่อ
    driver = GraphDatabase.driver(uri, auth=(user, password))
    driver.verify_connectivity()
    print("\n🎉 ไชโย! เชื่อมต่อสำเร็จแล้ว")
    print("✅ ตอนนี้คุณสามารถกลับไปรันช่อง Upload ข้อมูล (ช่องสุดท้าย) ได้เลยครับ!")
    driver.close()
except Exception as e:
    print(f"\n❌ ยังไม่ได้: {e}")
    if "Tv_SEC_ERROR_UNKNOWN_ISSUER" in str(e) or "Routing" in str(e):
         print("👉 ปัญหาน่าจะอยู่ที่ Network Firewall ของเน็ตบ้านบล็อกพอร์ต 7687 ครับ")

📍 กำลังลองเชื่อมต่อแบบผ่อนปรน (SSC)...
🔗 URI: neo4j+ssc://074cd7e1.databases.neo4j.io

❌ ยังไม่ได้: Failed to DNS resolve address 074cd7e1.databases.neo4j.io:7687: [Errno 11001] getaddrinfo failed
